In [1]:
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModel
import torch
from chromadb import Client, Settings
import chromadb
import sqlite3
from datetime import datetime
import numpy as np

# 初始化 MiniCPM 模型
path = "openbmb/MiniCPM3-4B"
device = "cuda"
tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    path, torch_dtype=torch.bfloat16, device_map=device, trust_remote_code=True
)

# 初始化向量模型
embed_tokenizer = AutoTokenizer.from_pretrained(
    "sentence-transformers/all-MiniLM-L6-v2"
)
embed_model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

if torch.cuda.is_available():
    embed_model = embed_model.cuda()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [2]:
# 在当前工作目录下创建chroma_db目录
chroma_db_path = os.path.join(os.getcwd(), "chroma_db")
if not os.path.exists(chroma_db_path):
    os.makedirs(chroma_db_path)

chroma_client = chromadb.PersistentClient(path=chroma_db_path)
collection = chroma_client.get_or_create_collection(name="chat_history")

# 初始化 SQLite 数据库
conn = sqlite3.connect("chat_history.db")
cursor = conn.cursor()
cursor.execute(
    """
CREATE TABLE IF NOT EXISTS chat_history (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_input TEXT,
    ai_response TEXT,
    timestamp DATETIME
)
"""
)
conn.commit()

def get_embedding(text: str) -> list:
    """获取文本的向量表示"""
    inputs = embed_tokenizer(
        text, return_tensors="pt", padding=True, truncation=True, max_length=512
    )
    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}

    with torch.no_grad():
        outputs = embed_model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
    return embeddings[0].tolist()

def add_chat_history(user_input: str, ai_response: str):
    """存储对话历史到SQLite"""
    try:
        cursor.execute("""
        INSERT INTO chat_history (
            user_input,
            ai_response,
            timestamp
        ) VALUES (?, ?, ?)
        """, (
            user_input,
            ai_response,
            datetime.now()
        ))
        conn.commit()
        return True
    except Exception as e:
        print(f"存储聊天历史时出错: {str(e)}")
        conn.rollback()
        return False

In [3]:
def split_counseling_text(text: str, chunk_size: int = 150, overlap: int = 50) :
    """专门针对心理咨询内容的文本分割

    Args:
        text: 要分割的文本
        chunk_size: 每个块的目标大小
        overlap: 重叠部分的大小，用于保持上下文连贯性
    """
    def is_sentence_boundary(text: str, pos: int) -> bool:
        """判断是否是自然的句子边界"""
        if pos >= len(text):
            return True

        # 标点符号表示句子结束
        end_marks = ['。', '！', '？', '…', '.', '!', '?']
        if any(text[pos] == mark for mark in end_marks):
            return True

        # 表示说话或情绪的标点
        speech_marks = ['：', ':', '"', '"', ''', ''']
        if any(text[pos] == mark for mark in speech_marks):
            return False

        return False

    def find_best_split_position(text: str, target_pos: int) -> int:
        """寻找最佳分割点"""
        # 在目标位置前后寻找最近的句子边界
        left = right = target_pos

        # 向左搜索
        while left > 0 and not is_sentence_boundary(text, left):
            left -= 1

        # 向右搜索
        while right < len(text) and not is_sentence_boundary(text, right):
            right += 1

        # 选择距离目标位置最近的边界
        if target_pos - left <= right - target_pos and left > 0:
            return left + 1
        return right + 1

    chunks = []
    start = 0
    text_length = len(text)

    while start < text_length:
        # 确定当前块的结束位置
        end = min(start + chunk_size, text_length)

        if end < text_length:
            # 寻找最佳分割点
            end = find_best_split_position(text, end)

        # 提取当前块
        current_chunk = text[start:end].strip()
        if current_chunk:
            chunks.append(current_chunk)

        # 计算下一个块的起始位置（考虑重叠）
        start = max(start + chunk_size - overlap, end - overlap)

    return chunks

In [4]:
def add_to_knowledge_base(content: str, chunk_size: int = 150):
    """向知识库添加心理咨询内容"""
    try:
        # 使用专门的分割方法
        chunks = split_counseling_text(content, chunk_size=chunk_size)

        print(f"\n文本被分割成 {len(chunks)} 个块:")
        for i, chunk in enumerate(chunks, 1):
            print(f"\n块 {i}:")
            print(chunk)

            try:
                embedding = get_embedding(chunk)
                collection.add(
                    documents=[chunk],
                    embeddings=[embedding],
                    ids=[str(len(collection.get()["ids"]) + 1)]
                )
            except Exception as e:
                print(f"处理块 {i} 时出错: {str(e)}")
                continue

        return {
            'status': 'success',
            'chunks_processed': len(chunks)
        }

    except Exception as e:
        print(f"添加到知识库时出错: {str(e)}")
        return {
            'status': 'error',
            'error': str(e)
        }

def check_knowledge_base():
    """检查知识库内容"""
    print("\n=== ChromaDB 知识库内容 ===")
    try:
        results = collection.get()
        print(f"ChromaDB 中共有 {len(results['documents'])} 条记录")
        for i, (doc, id) in enumerate(zip(results['documents'], results['ids']), 1):
            print(f"\n文档 {i} (ID: {id}):")
            print(doc)
    except Exception as e:
        print(f"获取 ChromaDB 内容时出错: {str(e)}")

    print("\n=== SQLite 对话历史 ===")
    try:
        cursor.execute("""
            SELECT id, timestamp, user_input, ai_response
            FROM chat_history
            ORDER BY timestamp DESC
        """)
        rows = cursor.fetchall()
        print(f"SQLite 中共有 {len(rows)} 条对话记录")
        for row in rows:
            print(f"\n对话ID: {row[0]}")
            print(f"时间: {row[1]}")
            print(f"用户: {row[2]}")
            print(f"AI: {row[3]}")
    except Exception as e:
        print(f"获取 SQLite 内容时出错: {str(e)}")

def search_similar_from_chromadb(query: str, similarity_threshold: float = 0.7, limit: int = 10) -> list:
    """从ChromaDB搜索语义相似的对话记录，并显示相似度

    Args:
        query: 查询文本
        similarity_threshold: 相似度阈值
        limit: 返回结果数量限制
    """
    query_embedding = get_embedding(query)
    results = collection.query(query_embeddings=[query_embedding], n_results=limit)

    semantic_matches = []
    print("\n=== 相似度排序结果 ===")
    if results["documents"]:
        distances = results.get("distances", [[]])[0]
        for i, (doc, distance) in enumerate(zip(results["documents"][0], distances), 1):
            # 使用余弦相似度计算
            doc_embedding = get_embedding(doc)
            similarity = np.dot(query_embedding, doc_embedding) / (
                np.linalg.norm(query_embedding) * np.linalg.norm(doc_embedding)
            )

            print(f"\n文档 {i}:")
            print(f"相似度: {similarity:.4f}")
            print(f"内容: {doc}")

            if similarity >= similarity_threshold:
                semantic_matches.append({
                    'content': doc,
                    'similarity': similarity
                })

    # 按相似度排序
    semantic_matches.sort(key=lambda x: x['similarity'], reverse=True)
    return semantic_matches

def get_recent_history_from_sqlite(limit: int = 5) -> list:
    """从SQLite获取最近的对话历史

    Args:
        limit: 返回的历史记录数量

    Returns:
        list: 最近的对话记录列表
    """
    cursor.execute(
        """
    SELECT user_input, ai_response FROM chat_history
    ORDER BY timestamp DESC LIMIT ?
    """, (limit,)
    )
    return [f"问：{row[0]}\n答：{row[1]}" for row in cursor.fetchall()]

def search_relevant(query: str, similarity_threshold: float = 0.7, limit: int = 10) -> tuple:
    """搜索相似的历史对话，同时从ChromaDB和SQLite获取结果"""
    semantic_matches = search_similar_from_chromadb(query, similarity_threshold, limit)
    recent_history = get_recent_history_from_sqlite()

    print("\n=== 最终筛选结果 ===")
    print(f"\n高于阈值({similarity_threshold})的匹配结果:")
    for i, match in enumerate(semantic_matches, 1):
        print(f"\n匹配 {i}:")
        print(f"相似度: {match['similarity']:.4f}")
        print(f"内容: {match['content']}")

    print("\n最近对话历史:")
    for i, history in enumerate(recent_history, 1):
        print(f"\n历史记录 {i}:")
        print(history)

    return semantic_matches, recent_history

In [5]:
def chat_with_memory():
    """对话主循环"""
    print("开始对话（输入'exit'结束）...")
    history = []

    while True:
        user_input = input("\n用户: ")
        if user_input.lower() in ["exit", "quit", "bye"]:
            break

        try:
            # 搜索相关历史记忆和最近对话
            semantic_matches, recent_history = search_relevant(user_input)

            # 构建提示
            prompt = user_input
            context_parts = []

            if semantic_matches:
                context_parts.append("相关历史对话：\n" + "\n".join(semantic_matches))
            if recent_history:
                context_parts.append("最近对话记录：\n" + "\n".join(recent_history))

            if context_parts:
                context = "\n\n".join(context_parts)
                prompt = f"{context}\n\n当前问题：{user_input}"
                print("\n" + context)

            # 生成回答
            response, history = model.chat(tokenizer, prompt, history=history)

            # 存储完整对话记录以及相关历史记忆
            add_chat_history(user_input, response)
            add_to_knowledge_base(response)

            print("\nAI:", response)

        except Exception as e:
            print(f"\n发生错误: {str(e)}")
            continue

if __name__ == "__main__":
    # 开始对话
    chat_with_memory()


开始对话（输入'exit'结束）...

用户: 我是小明


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48



=== 相似度排序结果 ===

=== 最终筛选结果 ===

高于阈值(0.7)的匹配结果:

最近对话历史:

文本被分割成 1 个块:

块 1:
你好，小明，很高兴认识你！如果是为了解决某个问题或寻求建议，请随时告诉我你需要的信息，我会尽我所能提供帮助。如果有任何疑问，也需要随时向我提问。

AI: 你好，小明，很高兴认识你！如果是为了解决某个问题或寻求建议，请随时告诉我你需要的信息，我会尽我所能提供帮助。如果有任何疑问，也需要随时向我提问。

用户: 我今年18岁


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



=== 相似度排序结果 ===

文档 1:
相似度: 0.5554
内容: 你好，小明，很高兴认识你！如果是为了解决某个问题或寻求建议，请随时告诉我你需要的信息，我会尽我所能提供帮助。如果有任何疑问，也需要随时向我提问。

=== 最终筛选结果 ===

高于阈值(0.7)的匹配结果:

最近对话历史:

历史记录 1:
问：我是小明
答：你好，小明，很高兴认识你！如果是为了解决某个问题或寻求建议，请随时告诉我你需要的信息，我会尽我所能提供帮助。如果有任何疑问，也需要随时向我提问。

最近对话记录：
问：我是小明
答：你好，小明，很高兴认识你！如果是为了解决某个问题或寻求建议，请随时告诉我你需要的信息，我会尽我所能提供帮助。如果有任何疑问，也需要随时向我提问。

文本被分割成 4 个块:

块 1:
了解到小明你今年18岁，正值青春年华，这是一个充满活力与可能的年纪。18岁意味着你已经成年，可以为自己做出更多的选择和决定。这是一个重要的成长阶段，你可以开始规划自己的未来，追求梦想，同时也要学会承担责任。

在这个年纪，你可能在学校接受教育，为将来的职业生涯做准备，或者你可能已经开始工作，积累社会经验。

块 2:
在这个年纪，你可能在学校接受教育，为将来的职业生涯做准备，或者你可能已经开始工作，积累社会经验。无论你选择哪条路，都要记得保持学习的心态，不断探索自我，挑战自我。

同时，18岁也是建立人际关系的黄金时期，你可以结交志同道合的朋友，拓展社交圈，这将对你的个人成长产生积极影响。

块 3:
黄金时期，你可以结交志同道合的朋友，拓展社交圈，这将对你的个人成长产生积极影响。不要害怕尝试新事物，勇敢地追求自己的兴趣和激情，这将让你的生活更加丰富多彩。

最后，记得保持健康的生活方式，关注身心健康，这是你成长的基石。祝你在18岁这一年，以及未来的日子里，能够勇敢追梦，活出精彩！

块 4:
康，这是你成长的基石。祝你在18岁这一年，以及未来的日子里，能够勇敢追梦，活出精彩！

AI: 了解到小明你今年18岁，正值青春年华，这是一个充满活力与可能的年纪。18岁意味着你已经成年，可以为自己做出更多的选择和决定。这是一个重要的成长阶段，你可以开始规划自己的未来，追求梦想，同时也要学会承担责任。

在这个年纪，你可能在学校接受教育，为将来的职业生涯做准备，或者你可能已经

...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



=== 相似度排序结果 ===

文档 1:
相似度: 0.6344
内容: 康，这是你成长的基石。祝你在18岁这一年，以及未来的日子里，能够勇敢追梦，活出精彩！

文档 2:
相似度: 0.5814
内容: 你好，小明，很高兴认识你！如果是为了解决某个问题或寻求建议，请随时告诉我你需要的信息，我会尽我所能提供帮助。如果有任何疑问，也需要随时向我提问。

文档 3:
相似度: 0.5196
内容: 你好，你只是匿名发了一个“我是小明”，并没有提供你的全名。如果你想分享你的全名，我可以帮你设定一个与你全名相关的个性签名；如果你不想分享，我也可以帮你设定一个匿名个性签名。请告诉我你的选择。

文档 4:
相似度: 0.5437
内容: 了解到小明你今年18岁，正值青春年华，这是一个充满活力与可能的年纪。18岁意味着你已经成年，可以为自己做出更多的选择和决定。这是一个重要的成长阶段，你可以开始规划自己的未来，追求梦想，同时也要学会承担责任。

在这个年纪，你可能在学校接受教育，为将来的职业生涯做准备，或者你可能已经开始工作，积累社会经验。

文档 5:
相似度: 0.5320
内容: 在这个年纪，你可能在学校接受教育，为将来的职业生涯做准备，或者你可能已经开始工作，积累社会经验。无论你选择哪条路，都要记得保持学习的心态，不断探索自我，挑战自我。

同时，18岁也是建立人际关系的黄金时期，你可以结交志同道合的朋友，拓展社交圈，这将对你的个人成长产生积极影响。

文档 6:
相似度: 0.5035
内容: 黄金时期，你可以结交志同道合的朋友，拓展社交圈，这将对你的个人成长产生积极影响。不要害怕尝试新事物，勇敢地追求自己的兴趣和激情，这将让你的生活更加丰富多彩。

最后，记得保持健康的生活方式，关注身心健康，这是你成长的基石。祝你在18岁这一年，以及未来的日子里，能够勇敢追梦，活出精彩！

=== 最终筛选结果 ===

高于阈值(0.7)的匹配结果:

最近对话历史:

历史记录 1:
问：我叫什么名字
答：你好，你只是匿名发了一个“我是小明”，并没有提供你的全名。如果你想分享你的全名，我可以帮你设定一个与你全名相关的个性签名；如果你不想分享，我也可以帮你设定一个匿名个性签名。请告诉我你的选择。

历史记录 2:
问：我今年18岁
答：了解到小明你今年18岁，正值青春年华，

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



=== 相似度排序结果 ===

文档 1:
相似度: 0.6990
内容: 你好，小明，很高兴认识你！如果是为了解决某个问题或寻求建议，请随时告诉我你需要的信息，我会尽我所能提供帮助。如果有任何疑问，也需要随时向我提问。

文档 2:
相似度: 0.6250
内容: 你好，小明！从你的对话记录来看，你今年18岁。如果你还有其他问题或需要帮助，随时告诉我，我会很乐意帮助你。

文档 3:
相似度: 0.5341
内容: 康，这是你成长的基石。祝你在18岁这一年，以及未来的日子里，能够勇敢追梦，活出精彩！

文档 4:
相似度: 0.4688
内容: 黄金时期，你可以结交志同道合的朋友，拓展社交圈，这将对你的个人成长产生积极影响。不要害怕尝试新事物，勇敢地追求自己的兴趣和激情，这将让你的生活更加丰富多彩。

最后，记得保持健康的生活方式，关注身心健康，这是你成长的基石。祝你在18岁这一年，以及未来的日子里，能够勇敢追梦，活出精彩！

文档 5:
相似度: 0.4657
内容: 你好，你只是匿名发了一个“我是小明”，并没有提供你的全名。如果你想分享你的全名，我可以帮你设定一个与你全名相关的个性签名；如果你不想分享，我也可以帮你设定一个匿名个性签名。请告诉我你的选择。

文档 6:
相似度: 0.4576
内容: 在这个年纪，你可能在学校接受教育，为将来的职业生涯做准备，或者你可能已经开始工作，积累社会经验。无论你选择哪条路，都要记得保持学习的心态，不断探索自我，挑战自我。

同时，18岁也是建立人际关系的黄金时期，你可以结交志同道合的朋友，拓展社交圈，这将对你的个人成长产生积极影响。

文档 7:
相似度: 0.4177
内容: 了解到小明你今年18岁，正值青春年华，这是一个充满活力与可能的年纪。18岁意味着你已经成年，可以为自己做出更多的选择和决定。这是一个重要的成长阶段，你可以开始规划自己的未来，追求梦想，同时也要学会承担责任。

在这个年纪，你可能在学校接受教育，为将来的职业生涯做准备，或者你可能已经开始工作，积累社会经验。

=== 最终筛选结果 ===

高于阈值(0.7)的匹配结果:

最近对话历史:

历史记录 1:
问：我今年多少岁
答：你好，小明！从你的对话记录来看，你今年18岁。如果你还有其他问题或需要帮助，随时告诉我，我会很乐意帮助你。

历史记

KeyboardInterrupt: Interrupted by user